In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agentic-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agentic-RAG


In [3]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

/tmp/ipykernel_51975/2001628515.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(k=6)


In [4]:
# 검색 결과를 가져옵니다.
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

[{'title': '분당 카카오프렌즈 판교 아지트 주차정보 카카오 캐릭터 굿즈샵',
  'url': 'https://blog.naver.com/dayslog/223559550059',
  'content': '오늘은 어, 분당 판교다! \u200b. 카카오프렌즈 판교아지트점. 경기도 성남시 분당구 판교역로 166 카카오판교아지트 1층 카카오프렌즈. 저장. 전화. 이 블로그',
  'score': 0.86024034},
 {'title': '[판교] 카카오 본사에 있는 카카오프렌즈샵 구경 : 네이버 블로그',
  'url': 'https://blog.naver.com/kimsso333/223186761448',
  'content': '... 카카오프렌즈샵을 소개해드려요! \u200b. \u200b. \u200b. 판교 카카오프렌즈샵 ... 아지트점. 경기도 성남시 분당구 판교역로 166 카카오판교아지트 1층 카카오프렌즈.',
  'score': 0.8524574},
 {'title': '리테일매거진 [라이프스타일숍] 카카오프렌즈 판교아지트점',
  'url': 'https://www.retailing.co.kr/article/a_view.php?art_idx=4697',
  'content': '회사명 : 코카리테일인사이트\n   대표 : 허영재\n   주소 : 서울시 마포구 서강로 60 유니세프빌딩 8층\n   tel : 02-522-1271~4\n   fax : 02-522-1275\n   E-mail : kocarm@naver.com\n   사업자 등록번호 : 486-81-00770\n   통신판매업신고번호 : 서울마포 제3484호\n   Copyright © RETAIL MAGAZINE All Rights Reserved.\n\nDesign by HIHOMPY [...] 리테일매거진;)\n       매거진 기사 PDF;)\n       유통 전문도서;)\n       보고서 & 통계자료;)\n       데이터;)\n\n   ##### event\n

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

In [7]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")

[Document(id='5e7a8dcb-707e-41ab-9943-acdf5dc04490', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'author': 'dj', 'moddate': '2023-12-08T13:28:38+09:00', 'pdfversion': '1.4', 'source': '../data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'page': 12, 'page_label': '13'}, page_content='SPRi AI Brief |  \n2023-12 월호\n10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개\nn삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스 ’를 공개\nn삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원\nn삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스 ’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 \n단계적으로

In [8]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

In [9]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [11]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

In [12]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [13]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [14]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'title': '프로야구 2024 포스트시즌 일정과 진행 방식 정리(각 팀별 포스트시즌 ...', 'url': 'https://pathfollower.tistory.com/entry/%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-2024-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EC%9D%BC%EC%A0%95%EA%B3%BC-%EC%A7%84%ED%96%89-%EB%B0%A9%EC%8B%9D-%EC%A0%95%EB%A6%AC%EA%B0%81-%ED%8C%80%EB%B3%84-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EC%A7%84%EC%B6%9C-%ED%99%95%EB%A5%A0', 'content': '플레이오프는 준플레이오프 승리팀과 정규시즌 2위 팀이 맞붙는 대결이다. 5판 3선승으로 진행된다.  \n1,2,5경기는 정규리그 2위 팀 홈구장에서 열리며 3,4경기는 준플레이오프 승리팀 홈구장에서 열린다.  \n   \n2024년 프로야구 플레이오프 일정은 다음과 같다.\n\n 10월 13일 (일) 14:00 - 대구\n 10월 14일 (월) 18:30 - 대구\n 10월 16일 (수) 18:30 - 준플레이오프 승리 팀 홈구장\n 10월 17일 (목) 18:30 - 준플레이오프 승리 팀 홈구장\n 10월 19일 (토) 14:00 - 대구\n\n2024년 9월 17일 기준으로 플레이오프 경기는 두산, KT, LG 중 한 팀과 삼성이 하게 될 것이 확실시된다.\n\n### 프로야구 2024 포스트시즌 진행 방식 - 4. 한국시리즈 [